In [1]:
import pandas as pd

df=pd.read_excel('food production and global warming.xls')
abstract=df['Abstract']
abstract = abstract.astype(str)

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora
import re

lemmatizer = WordNetLemmatizer()
def clean_text(text):

    # 특수 문자 및 숫자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 소문자 변환
    text = text.lower()

    # 토큰화
    tokens = nltk.word_tokenize(text)

    # 불용어 제거
    s_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in s_words]

    # 표제어 추출
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # 토큰 길이가 2보다 작은 경우(불용어) 제거
    tokens = [token for token in tokens if len(token) > 2]
    
    return tokens

cleaned_abstract = abstract.apply(clean_text)
cleaned_abstract

dic = corpora.Dictionary(cleaned_abstract)
# 빈도수가 너무 낮거나 높은 단어 제거
dic.filter_extremes(no_below=5, no_above=0.5)
# corpus 생성, 각 문서를 단어 출현 빈도로 변환하여 corpus 생성
corpus = [dic.doc2bow(text) for text in cleaned_abstract]

In [4]:
from gensim import models
from gensim.models import CoherenceModel

model_lda = models.LdaModel(corpus, num_topics=10, id2word=dic, passes=10, random_state=0)

coherence_model_lda = CoherenceModel(model=model_lda, texts=cleaned_abstract, dictionary=dic, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print("Coherence Score:", coherence_score)

Coherence Score: 0.47733502030954805


In [5]:
for topic_num, topics in model_lda.print_topics(-1):
    print(f'토픽: {topic_num}')
    print(f'토픽: {topics}')
    print('\n')

토픽: 0
토픽: 0.020*"fuel" + 0.015*"energy" + 0.014*"biodiesel" + 0.013*"biofuels" + 0.012*"product" + 0.012*"environmental" + 0.011*"oil" + 0.009*"fossil" + 0.009*"chemical" + 0.008*"microalgae"


토픽: 1
토픽: 0.024*"temperature" + 0.022*"effect" + 0.014*"specie" + 0.013*"change" + 0.011*"ecosystem" + 0.010*"increase" + 0.010*"marine" + 0.009*"climate" + 0.008*"web" + 0.007*"community"


토픽: 2
토픽: 0.020*"water" + 0.014*"change" + 0.012*"ocean" + 0.011*"temperature" + 0.010*"increase" + 0.010*"carbon" + 0.010*"animal" + 0.010*"sea" + 0.008*"web" + 0.007*"fish"


토픽: 3
토픽: 0.028*"emission" + 0.012*"carbon" + 0.011*"gas" + 0.011*"energy" + 0.010*"system" + 0.010*"greenhouse" + 0.009*"potential" + 0.008*"ghg" + 0.007*"crop" + 0.007*"soil"


토픽: 4
토픽: 0.013*"model" + 0.011*"scenario" + 0.010*"system" + 0.009*"climate" + 0.008*"temperature" + 0.008*"effect" + 0.006*"environmental" + 0.006*"change" + 0.006*"result" + 0.006*"degree"


토픽: 5
토픽: 0.014*"system" + 0.011*"waste" + 0.009*"size" + 0.009*"